In [ ]:
import math
import numpy as np
import pandas as pd
from pandas import Series
#
from numpy import log, mean, sqrt, where, std, exp, sign
# from scipy import linalg as LA
# from scipy.optimize import curve_fit
# from sklearn.linear_model import LinearRegression
#
import matplotlib.pyplot as plt
import time
#
from scipy import stats
from scipy.stats import entropy
from scipy.stats import vonmises
#

In [ ]:
#
# read sample files
#
fps = 25
dt = 1.0/fps
r_contact = 2.0
#
readfile_1 = "sample-traj-ballistic.txt"
print ("read ", readfile_1)
#
readfile_2 = "sample-traj-confined.txt"
print ("read ", readfile_2)
#
readfile_3 = "sample-traj-subballistic.txt"
print ("read ", readfile_3)
#
columns = ['frame', 'delta_x', 'delta_y']
#
traj_ballistic    = pd.read_csv(readfile_1, sep= "\t", names = columns) 
traj_confined     = pd.read_csv(readfile_2, sep= "\t", names = columns) 
traj_subballistic = pd.read_csv(readfile_3, sep= "\t", names = columns) 

In [ ]:
# convert two pandas columns into tuples
position_ballistic    = list(zip(traj_ballistic.delta_x, traj_ballistic.delta_y))
position_confined     = list(zip(traj_confined.delta_x, traj_confined.delta_y))
position_subballistic = list(zip(traj_subballistic.delta_x, traj_subballistic.delta_y))

In [ ]:
x = [traj_ballistic.delta_x, traj_confined.delta_x, traj_subballistic.delta_x]
y = [traj_ballistic.delta_y, traj_confined.delta_y, traj_subballistic.delta_y]
n_datapoints = [len(x[0]), len(x[1]), len(x[2])]
print(n_datapoints)

In [ ]:
def generate_circle(x0, y0, r_circle):
    theta_circle = np.linspace(0, 2*np.pi, 300)
    x_circle = x0+r_circle*np.cos(theta_circle)
    y_circle = y0+r_circle*np.sin(theta_circle) 
    return x_circle, y_circle

In [ ]:
#
# basic functions
#
def get_displacement(x1, y1, x2, y2):
    #
    # input x1: float, first x coordinate
    # input y1: float, first y coordinate
    # input x2: float, second x coordinate
    # input y2: float, second y coordinate
    # return: float, displacement between two points
    #
    return sqrt((x1-x2)**2+(y1-y2)**2)

def get_displacements(x, y, n_datapoints):
    #
    # input x: array, list of x coordinate
    # input y: array, list of y coordinate
    # return: array, list of displacements between x and y coordinates
    #
    return np.array([get_displacement(x[i], y[i], x[i-1], y[i-1]) for i in range(1, n_datapoints)])
#
# feature 1: efficiency and straightness
#
# efficiency relates the net squared displacement of a particle to the sum of squared step lengths
# measure for linearity of trajectory
# like asymmetry, it may help to detect directed motion
# return: float, efficiency parameter
#
def get_efficiency(x, y, n_datapoints):
    upper = get_displacement(x[n_datapoints-1], y[n_datapoints-1], x[0], y[0])**2
    displacements_to_square = get_displacements(x, y, n_datapoints)**2
    lower = (n_datapoints-1)*sum(displacements_to_square)
    efficiency = upper/lower
    return efficiency

#
# Straightness
# a measure of the average direction change between subsequent steps.
# return: float, straing
#
def get_straightness(x, y, n_datapoints):
    upper = get_displacement(x[n_datapoints-1], y[n_datapoints-1], x[0], y[0])
    displacements = np.array([get_displacement(x[i], y[i], x[i-1], y[i-1]) for i in range(1, n_datapoints)])
    lower = sum(displacements)
    straightness = upper/lower
    return straightness

In [ ]:
#
# see:
# Liu et al. PRE 2017
# Establishing the kinetics of ballistic-to-diffusive transition using directional statistics
# Appendix A: Determining theta from trajectory
#
def compute_turning_angle(traj_x, traj_y, tau_frame, binwidth):
    #
    # compute turning angle
    #    
    # traj_x = record_raw['relative_x']
    # traj_y = record_raw['relative_y']
    traj_x = pd.Series(traj_x)
    traj_y = pd.Series(traj_y)
    relative_x = traj_x[::tau_frame] # every tau-th row
    relative_y = traj_y[::tau_frame] # every tau-th row  
    relative_x = relative_x.reset_index(drop=True) # reset row index
    relative_y = relative_y.reset_index(drop=True) # reset row index
    turning_angle = []
    for i in range (1, len(relative_x)-1):
        # print(i)
        diff_x1 = relative_x[i]-relative_x[i-1]
        diff_x2 = relative_x[i+1]-relative_x[i]
        diff_y1 = relative_y[i]-relative_y[i-1]
        diff_y2 = relative_y[i+1]-relative_y[i] 
        # compute k1 and k2
        k1 = 0 # diff_x1 > 0 and diff_y1 > 0
        if (diff_x1 >= 0):
            if (diff_y1 >= 0):
                k1 = 0
            else:
                k1 = 2
        if (diff_x1 < 0):
            k1 = 1
        k2 = 0
        if (diff_x2 >= 0):
            if (diff_y2 >= 0):
                k2 = 0
            else:
                k2 = 2
        if (diff_x2 < 0):
            k2 = 1    
        # compute phi_1 and phi_2
        # arc tangent of y/x in radians
        phi_1 = k1*np.pi + math.atan2(diff_y1, diff_x1) # 0, ..., 2*np.pi
        phi_2 = k2*np.pi + math.atan2(diff_y2, diff_x2) # 0, ..., 2*np.pi
        # compute m
        m = 0
        phi_diff = abs(phi_2-phi_1)
        if (phi_diff < np.pi):
            m = 0
        if (phi_diff > np.pi):
            if (phi_2 > phi_1):
                m = -1
            if (phi_2 < phi_1):
                m = 1
        # compute theta
        theta_i = 2*m*np.pi+phi_2-phi_1  # -np.pi, ..., np.pi
        turning_angle.append(theta_i)
    #
    # compute entropy
    #
    epsilon_p = 0.0001
    epsilon_q = 0.0001
    # create relative frequency histogram
    turning_angle_deg = [(x/np.pi)*180.0 for x in turning_angle] # degree
    x_max = 180 # deg
    x_min = -180 # deg
    #
    bin_list = np.arange(x_min, x_max, binwidth) 
    hist_p, edges_p = np.histogram(turning_angle_deg, bins=bin_list)
    # print(len(bin_list), len(edges_p), len(hist_p))
    freq_p = hist_p/float(hist_p.sum())
    freq_p += epsilon_p
    pk = np.reshape(freq_p, -1)
    base = len(bin_list) # normalized entropy
    entropy_p = entropy(pk, base=base)        
    #    
    #
    return turning_angle, turning_angle_deg, entropy_p

In [ ]:
#
# prepare subplots
#
n_columns = 3
n_rows = 1
#
subplot_x = 5.0
subplot_y = 4.0
#
figsize_x = subplot_x*n_columns
figsize_y = subplot_y*n_rows
fig = plt.figure(figsize=(figsize_x, figsize_y))      
#    
x_circle, y_circle = generate_circle(0.0, 0.0, r_contact)
#
for j in range (0, n_columns):
    ax = fig.add_subplot(n_rows,n_columns,(j+1))
    plt.scatter(0.0, 0.0, marker = 'x', s = 50, color='k',linewidths=2)
    plt.plot(x[j], y[j], color = 'b', lw=3) 
    plt.plot(x_circle, y_circle, 'r--', alpha=0.5, lw=2) 
    #
    plt.xticks(np.arange(-4, 4.05, 1.0))
    plt.yticks(np.arange(-4, 4.05, 1.0))
    plt.xlim(-2.5, 2.5)
    plt.ylim(-2.5, 2.5)
    plt.tick_params(labelsize=12)
    plt.xlabel("position x (m)", fontsize=12)
    plt.ylabel("position y (m)", fontsize=12)         
# set the spacing between subplots
plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)    
plt.show()  

In [ ]:
#
binwidth_turning_angle = 15 # deg
tau_turning_angle = 2 # frames
#
turning_angle_ballistic, turning_angle_deg_ballistic, turning_angle_entropy_ballistic = compute_turning_angle(traj_ballistic.delta_x, traj_ballistic.delta_y, tau_turning_angle, binwidth_turning_angle)
turning_angle_confined, turning_angle_deg_confined, turning_angle_entropy_confined = compute_turning_angle(traj_confined.delta_x, traj_confined.delta_y, tau_turning_angle, binwidth_turning_angle)
turning_angle_subballistic, turning_angle_deg_subballistic, turning_angle_entropy_subballistic = compute_turning_angle(traj_subballistic.delta_x, traj_subballistic.delta_y, tau_turning_angle, binwidth_turning_angle)
#
print(round(turning_angle_entropy_ballistic, 3))
print(round(turning_angle_entropy_confined, 3))
print(round(turning_angle_entropy_subballistic, 3))
turning_angle_deg = [turning_angle_deg_ballistic, turning_angle_deg_confined, turning_angle_deg_subballistic]

In [ ]:
#
n_columns = 3
n_rows = 1
subplot_x = 6.0
subplot_y = 3.0
figsize_x = subplot_x*n_columns
figsize_y = subplot_y*n_rows
fig = plt.figure(figsize=(figsize_x, figsize_y))    
#    
turning_angle_deg = [turning_angle_deg_ballistic, turning_angle_deg_confined, turning_angle_deg_subballistic]
turning_angle_max = 180.5
turning_angle_min = -180.0
turning_angle_binwidth = 15
binlist = np.arange(turning_angle_min, turning_angle_max, turning_angle_binwidth)
#
subplot_title = []
subplot_title.append("ballistic")
subplot_title.append("confined")
subplot_title.append("sub-ballistic")
#
for j in range (0, n_columns):
    ax = fig.add_subplot(1, n_columns, (j+1)) 
    counts = plt.hist(turning_angle_deg[j], bins=binlist, alpha=0.5, weights=np.ones_like(turning_angle_deg[j])/len(turning_angle_deg[j])) # alpha controls transparency
    #
    xtick_space = 45
    plt.xticks(np.arange(turning_angle_min, turning_angle_max, xtick_space))
    plt.tick_params(labelsize=12)
    plt.xlabel("turning angle (deg)", fontsize=14)
    plt.ylabel('relative frequency', fontsize=14)       
    plt.title(subplot_title[j], fontsize=16)
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)   
plt.show() 
#
print("ballistic\t{:.2f}".format(np.mean(turning_angle_deg[0]))+"\t{:.2f}".format(np.std(turning_angle_deg[0])))
print("confined\t{:.2f}".format(np.mean(turning_angle_deg[1]))+"\t{:.2f}".format(np.std(turning_angle_deg[1])))
print("subballistic\t{:.2f}".format(np.mean(turning_angle_deg[2]))+"\t{:.2f}".format(np.std(turning_angle_deg[2])))

In [ ]:
# efficiency
efficiency_ballistic    = get_efficiency(x[0], y[0], n_datapoints[0])
efficiency_confined     = get_efficiency(x[1], y[1], n_datapoints[1])
efficiency_subballistic = get_efficiency(x[2], y[2], n_datapoints[2])
#
print("type\tentropy\tefficiency")
print("ballistic\t{:.3f}".format(turning_angle_entropy_ballistic)+"\t{:.3f}".format(efficiency_ballistic))
print("confined\t{:.3f}".format(turning_angle_entropy_confined)+"\t{:.3f}".format(efficiency_confined))
print("subballistic\t{:.3f}".format(turning_angle_entropy_subballistic)+"\t{:.3f}".format(efficiency_subballistic))

In [ ]:
readfile_1a = "sample-traj-ballistic_65_99.txt"
print ("read ", readfile_1a)
#
readfile_2a = "sample-traj-confined_25_46.txt"
print ("read ", readfile_2a)
#
readfile_3a = "sample-traj-subballistic_463_566.txt"
print ("read ", readfile_3a)
#
columns = ['frame', 'x_i', 'y_i', 'x_j', 'y_j']
#
traj_ballistic_ij    = pd.read_csv(readfile_1a, sep= "\t", names = columns) 
traj_confined_ij     = pd.read_csv(readfile_2a, sep= "\t", names = columns) 
traj_subballistic_ij = pd.read_csv(readfile_3a, sep= "\t", names = columns) 

In [ ]:
#
# x_i = [traj_ballistic_ij.x_i, traj_confined_ij.x_i, traj_subballistic_ij.x_i]
# y_i = [traj_ballistic_ij.y_i, traj_confined_ij.y_i, traj_subballistic_ij.y_i]
# x_j = [traj_ballistic_ij.x_j, traj_confined_ij.x_j, traj_subballistic_ij.x_j]
# y_j = [traj_ballistic_ij.y_j, traj_confined_ij.y_j, traj_subballistic_ij.y_j]
#
x_i = [traj_ballistic_ij.x_i, traj_confined_ij.x_i, traj_subballistic_ij.y_i]
y_i = [traj_ballistic_ij.y_i, traj_confined_ij.y_i, traj_subballistic_ij.x_i]
x_j = [traj_ballistic_ij.x_j, traj_confined_ij.x_j, traj_subballistic_ij.y_j]
y_j = [traj_ballistic_ij.y_j, traj_confined_ij.y_j, traj_subballistic_ij.x_j]
n_datapoints = [len(x_i[0]), len(x_i[1]), len(x_i[2])]
print(n_datapoints)

In [ ]:
#
# prepare subplots
#
n_columns = 3
n_rows = 1
#
subplot_x = 5.0
subplot_y = 1.2
#
figsize_x = subplot_x*n_columns
figsize_y = subplot_y*n_rows
fig = plt.figure(figsize=(figsize_x, figsize_y))      
#
for j in range (0, n_columns):
    ax = fig.add_subplot(n_rows,n_columns,(j+1))
    plt.plot(x_i[j], y_i[j], 'b--', lw=2) 
    plt.plot(x_j[j], y_j[j], 'r', lw=2) # alpha=0.5, 
    plt.axis('off')      
# set the spacing between subplots
plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.5, hspace=0.5)    
plt.show()  